In [159]:
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow import keras
from keras import preprocessing
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers import Embedding, LSTM, Dense, Dropout, Input, concatenate


In [160]:
# Load your dataset (replace 'data.csv' with your file)
data = pd.read_csv('data.csv')

# Assuming your dataset has 'Type' and 'Issue' columns
X = data['Issue'].values
y = data['Type'].values


In [161]:
# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X_tokenized = tokenizer.texts_to_sequences(X)


In [162]:
# Padding sequences to have the same length
max_sequence_length = max(len(seq) for seq in X_tokenized)
X_padded = pad_sequences(X_tokenized, maxlen=max_sequence_length, padding='post')


In [163]:
# Vocabulary size
vocab_size = len(tokenizer.word_index) + 1

# One-hot encode the 'Type' labels
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
num_classes = len(label_encoder.classes_)

In [164]:
# Build the model
embedding_dim = 100
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length))
model.add(LSTM(128))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))


In [165]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_padded,y_encoded, batch_size=32, epochs=100)
model.save('issue_type_prediction_model.h5')

Epoch 1/100
2/2 [==============================] - 3s 28ms/step - loss: 2.1975 - accuracy: 0.1250
Epoch 2/100
2/2 [==============================] - 0s 20ms/step - loss: 2.1899 - accuracy: 0.1750
Epoch 3/100
2/2 [==============================] - 0s 22ms/step - loss: 2.1932 - accuracy: 0.1500
Epoch 4/100
2/2 [==============================] - 0s 20ms/step - loss: 2.1807 - accuracy: 0.1250
Epoch 5/100
2/2 [==============================] - 0s 21ms/step - loss: 2.1663 - accuracy: 0.2250
Epoch 6/100
2/2 [==============================] - 0s 23ms/step - loss: 2.1698 - accuracy: 0.1250
Epoch 7/100
2/2 [==============================] - 0s 19ms/step - loss: 2.1617 - accuracy: 0.2500
Epoch 8/100
2/2 [==============================] - 0s 21ms/step - loss: 2.1631 - accuracy: 0.1750
Epoch 9/100
2/2 [==============================] - 0s 21ms/step - loss: 2.1468 - accuracy: 0.2000
Epoch 10/100
2/2 [==============================] - 0s 21ms/step - loss: 2.0884 - accuracy: 0.2000
Epoch 11/100
2/2 [=

c:\Users\nithi\Desktop\Societe Generale\.venv\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [167]:
# Load the trained model
loaded_model = tf.keras.models.load_model('issue_type_prediction_model.h5')

# Preprocess user input
user_input = "firewall rules prevent users from accessing specific resources."
user_input_tokenized = tokenizer.texts_to_sequences([user_input])
user_input_padded = pad_sequences(user_input_tokenized, maxlen=max_sequence_length, padding='post')

# Predict the issue type
predicted_probabilities = loaded_model.predict(user_input_padded)
predicted_class = np.argmax(predicted_probabilities, axis=-1)
predicted_type = label_encoder.inverse_transform(predicted_class)[0]
print("Predicted Issue Type:", predicted_type)

1/1 [==============================] - 1s 581ms/step
Predicted Issue Type: Network Connectivity Issues

Users experience connectivity problems, affecting their ability to access resources.


In [168]:
X_issue = data['Issue'].values
X_type = data['Type'].values
y_resolution = data['Resolution'].values

In [169]:
X_issue_tokenized = tokenizer.texts_to_sequences(X_issue)
X_type_encoded = label_encoder.transform(X_type)

max_sequence_length = max(len(seq) for seq in X_issue_tokenized)
X_issue_padded = pad_sequences(X_issue_tokenized, maxlen=max_sequence_length, padding='post')
tokenizer_resolution = Tokenizer()
tokenizer_resolution.fit_on_texts(y_resolution)
y_resolution_tokenized = tokenizer_resolution.texts_to_sequences(y_resolution)
max_sequence_length_resolution = max(len(seq) for seq in y_resolution_tokenized)
y_resolution_padded = pad_sequences(y_resolution_tokenized, maxlen=max_sequence_length_resolution, padding='post')



In [170]:
from sklearn.preprocessing import LabelEncoder
resolution_label_encoder = LabelEncoder()
y_resolution_encoded = resolution_label_encoder.fit_transform(y_resolution)
num_resolution_classes = len(resolution_label_encoder.classes_)


In [171]:
embedding_dim = 100
lstm_units = 128

# Input layers
input_issue = Input(shape=(max_sequence_length,))
input_type = Input(shape=(1,))

# Embedding layer for Issue
embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length)(input_issue)
lstm_layer = LSTM(lstm_units)(embedding_layer)

In [172]:
concatenated = concatenate([lstm_layer, input_type])

# Dense layers for Resolution prediction
dense_layer = Dense(64, activation='relu')(concatenated)
dropout_layer = Dropout(0.5)(dense_layer)
output_resolution = Dense(num_resolution_classes, activation='softmax')(dropout_layer)


In [173]:
# Define the model
combined_model = Model(inputs=[input_issue, input_type], outputs=output_resolution)

# Compile the model
combined_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# ... (previous code)

# Train the model
combined_model.fit(
    [X_padded, X_type_encoded],
    y_resolution_encoded,
    batch_size=8,
    epochs=200
)
combined_model.save('issue_resolution_model.h5')


Epoch 1/200
5/5 [==============================] - 4s 24ms/step - loss: 3.8089 - accuracy: 0.0000e+00
Epoch 2/200
5/5 [==============================] - 0s 22ms/step - loss: 3.8246 - accuracy: 0.0500
Epoch 3/200
5/5 [==============================] - 0s 21ms/step - loss: 3.7829 - accuracy: 0.0000e+00
Epoch 4/200
5/5 [==============================] - 0s 23ms/step - loss: 3.6918 - accuracy: 0.0250
Epoch 5/200
5/5 [==============================] - 0s 23ms/step - loss: 3.6636 - accuracy: 0.0500
Epoch 6/200
5/5 [==============================] - 0s 23ms/step - loss: 3.7094 - accuracy: 0.0250
Epoch 7/200
5/5 [==============================] - 0s 23ms/step - loss: 3.6867 - accuracy: 0.0250
Epoch 8/200
5/5 [==============================] - 0s 23ms/step - loss: 3.6481 - accuracy: 0.0500
Epoch 9/200
5/5 [==============================] - 0s 24ms/step - loss: 3.5966 - accuracy: 0.0750
Epoch 10/200
5/5 [==============================] - 0s 22ms/step - loss: 3.6717 - accuracy: 0.0000e+00
Epoch 1

c:\Users\nithi\Desktop\Societe Generale\.venv\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [174]:
combined_model = tf.keras.models.load_model('issue_resolution_model.h5')

user_input = "Data Format Incompatibility: Imported data is in a format not supported by the import process."
user_input_tokenized = tokenizer.texts_to_sequences([user_input])
user_input_padded = pad_sequences(user_input_tokenized, maxlen=max_sequence_length, padding='post')

# Predict Issue Type
predicted_probabilities_type = loaded_model.predict(user_input_padded)
predicted_class_type = np.argmax(predicted_probabilities_type, axis=-1)
predicted_type = label_encoder.inverse_transform(predicted_class_type)[0]

# Tokenize the user's Issue description
user_issue_tokenized = tokenizer.texts_to_sequences([user_input])
user_issue_padded = pad_sequences(user_issue_tokenized, maxlen=max_sequence_length, padding='post')

# Encode the predicted Issue Type
user_type_encoded = label_encoder.transform([predicted_type])

# Predict the Resolution
predicted_resolution_probs = combined_model.predict([user_issue_padded, user_type_encoded])
predicted_resolution_class = np.argmax(predicted_resolution_probs)
predicted_resolution = resolution_label_encoder.inverse_transform([predicted_resolution_class])[0]


print ('\n\n')
print ("User Issue:", user_input)
print ()
print ("Predicted Issue Type:", predicted_type)
print ()
print ("Predicted resolution:", predicted_resolution)

1/1 [==============================] - 1s 587ms/step



User Issue: Data Format Incompatibility: Imported data is in a format not supported by the import process.

Predicted Issue Type: Data Import Failure

Scheduled data import processes fail to transfer and update data.

Predicted resolution: AI system identifies data format mismatches, transforms data, and retries imports.
